In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import opendatasets as od
  
od.download("https://www.kaggle.com/datasets/kiraio/organizeflowersdataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sherifmagdyabdellah
Your Kaggle Key: ··········


100%|██████████| 514M/514M [00:06<00:00, 81.5MB/s]


In [3]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D,Activation
from keras.layers import Dense ,Conv2D,Flatten,Dropout,ReLU,Activation,Input
import os
import cv2
from keras import backend as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import numpy as np
import time
import random
random.seed(10)
np.random.seed(10)
tf.random.set_seed(10)

In [4]:
# variable
## image path
train_data_dir = '/content/organizeflowersdataset/change_flowers/change_flowers/train'
validation_data_dir = '/content/organizeflowersdataset/change_flowers/change_flowers/validation'
## other
img_width, img_height = 200, 200
nb_train_samples = 100
nb_validation_samples = 800
top_epochs = 50
fit_epochs = 50
batch_size = 24
nb_classes = 5
nb_epoch = 10

In [5]:
x_train=[]
y_train=[]
for subdir, dirs, files in os.walk(train_data_dir):
    for file in files:
        img = cv2.imread(os.path.join(subdir, file))
        x_train.append(cv2.resize(img,(200,200))/255.0) 
        y_train.append(subdir.split('/')[-1])

In [6]:
x_valid=[]
y_valid=[]
for subdir, dirs, files in os.walk(validation_data_dir):
    for file in files:
        img = cv2.imread(os.path.join(subdir, file))
        x_valid.append(cv2.resize(img,(200,200))/255.0) 
        y_valid.append(subdir.split('/')[-1])

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train)
y_valid=le.transform(y_valid)

In [8]:
from sklearn.utils import shuffle
x_valid, y_valid = shuffle(x_valid, y_valid)
x_train, y_train = shuffle(x_train, y_train)

In [9]:
x_train=np.array(x_train[0:1500])
y_train=y_train[0:1500]
x_valid=np.array(x_valid[0:500])
y_valid=y_valid[0:500]

In [10]:
def create_model():
  # import vgg16 model
  input_tensor = Input(shape=(img_width, img_height, 3))
  vgg16 = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_tensor=input_tensor)

  # creating an FC layer
  top_model = Sequential()
  top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
  top_model.add(Dense(256, activation='relu'))
  top_model.add(Dropout(0.5))
  top_model.add(Dense(nb_classes, name='dense_output'))
  top_model.add(Activation('softmax', dtype='float32', name='predictions'))

  # bound VGG 16 and FC layer
  vgg_model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

  # create model
  vgg_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer="adam",
                metrics=['accuracy']
  )

  return vgg_model


In [11]:
model_without_mixedPrecision=create_model()

58900480/58889256 [==============================] - 0s 0us/step


In [12]:
model_without_mixedPrecision.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [13]:
import time
training_start = time.time()
history_without_mixedPrecision = model_without_mixedPrecision.fit(
          x_train,y_train,batch_size=128,
          epochs=20,
          validation_data=(x_valid,y_valid),
          shuffle=True,
          verbose=1)
training_stop = time.time()
training_time = training_stop - training_start
print(f"Training time: {training_time}")

Epoch 1/20
12/12 [==============================] - 72s 4s/step - loss: 2.5768 - accuracy: 0.2333 - val_loss: 1.7135 - val_accuracy: 0.1200
Epoch 2/20
12/12 [==============================] - 19s 2s/step - loss: 2.0114 - accuracy: 0.2320 - val_loss: 1.6035 - val_accuracy: 0.1680
Epoch 3/20
12/12 [==============================] - 19s 2s/step - loss: 1.5712 - accuracy: 0.3187 - val_loss: 1.4133 - val_accuracy: 0.4480
Epoch 4/20
12/12 [==============================] - 19s 2s/step - loss: 1.4286 - accuracy: 0.4040 - val_loss: 1.3664 - val_accuracy: 0.4060
Epoch 5/20
12/12 [==============================] - 19s 2s/step - loss: 1.3061 - accuracy: 0.4567 - val_loss: 1.2444 - val_accuracy: 0.4540
Epoch 6/20
12/12 [==============================] - 19s 2s/step - loss: 1.1977 - accuracy: 0.4993 - val_loss: 1.1934 - val_accuracy: 0.5120
Epoch 7/20
12/12 [==============================] - 19s 2s/step - loss: 1.1611 - accuracy: 0.5153 - val_loss: 1.2216 - val_accuracy: 0.5020
Epoch 8/20
12/12 [==

In [14]:
import gc
gc.collect()

1434

In [15]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [16]:
model_mixedPrecision = create_model()

In [17]:
import time
training_start_1 = time.time()
history_model_mixedPrecision = model_mixedPrecision.fit(
          x_train,y_train,batch_size=128,
          epochs=20,
          validation_data=(x_valid,y_valid),
          shuffle=True,
          verbose=1)
training_stop_1 = time.time()
training_time_1 = training_stop_1 - training_start_1
print(f"Training time: {training_time_1}")

Epoch 1/20
12/12 [==============================] - 165s 7s/step - loss: 6.3886 - accuracy: 0.2220 - val_loss: 1.6071 - val_accuracy: 0.1200
Epoch 2/20
12/12 [==============================] - 10s 862ms/step - loss: 1.6333 - accuracy: 0.2587 - val_loss: 1.6008 - val_accuracy: 0.2280
Epoch 3/20
12/12 [==============================] - 10s 856ms/step - loss: 1.5731 - accuracy: 0.3227 - val_loss: 1.4765 - val_accuracy: 0.4300
Epoch 4/20
12/12 [==============================] - 10s 855ms/step - loss: 1.5561 - accuracy: 0.3267 - val_loss: 1.5027 - val_accuracy: 0.4400
Epoch 5/20
12/12 [==============================] - 10s 854ms/step - loss: 1.5335 - accuracy: 0.3387 - val_loss: 1.5164 - val_accuracy: 0.3700
Epoch 6/20
12/12 [==============================] - 10s 856ms/step - loss: 1.3585 - accuracy: 0.4280 - val_loss: 1.4188 - val_accuracy: 0.3180
Epoch 7/20
12/12 [==============================] - 10s 852ms/step - loss: 1.2741 - accuracy: 0.4553 - val_loss: 1.3084 - val_accuracy: 0.4880
E

after we train two models one by float32 precision and one with mixed precision we conclude that model with mixed precision faster than default double in each epoch (default model time per epoch =20 s
mixed precision 10 s per epoch )
and That will be much clearer in deeper model with huge train data